<a href="https://colab.research.google.com/github/IzzulGod/GPT2-Small-Indonesian/blob/main/source/fine_tune_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets peft accelerate
!pip install -U datasets fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 25.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 25.1 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("IzzulGod/GPT2-Small-Indonesian")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    "IzzulGod/GPT2-Small-Indonesian",
    torch_dtype=torch.float16,
    device_map="auto"
)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/569 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/470 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/768 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/498M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/119 [00:00<?, ?B/s]

In [3]:
from datasets import load_dataset

dataset = load_dataset("FreedomIntelligence/sharegpt-indonesian")

def convert_sharegpt_to_chat(example):
    messages = example["conversations"]
    chat_text = ""
    for message in messages:
        role = message["from"]
        content = message["value"]
        if content is None:
            continue
        if role == "human":
            chat_text += f"User: {content.strip()}\n"
        elif role == "gpt":
            chat_text += f"AI: {content.strip()}\n"
    return {"text": chat_text}

dataset = dataset.map(convert_sharegpt_to_chat)

def tokenize(example):
    tokens = tokenizer(
        example["text"],
        truncation=True,
        max_length=512,
        padding="max_length"
    )
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

tokenized_dataset = dataset.map(tokenize, batched=True, remove_columns=dataset["train"].column_names)

sample = dataset["train"][0]["text"]
tokenized_sample = tokenizer(sample)
print(tokenized_sample)
print(tokenizer.decode(tokenized_sample["input_ids"]))


README.md:   0%|          | 0.00/208 [00:00<?, ?B/s]

sharegpt-indonesian.json:   0%|          | 0.00/31.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/5560 [00:00<?, ? examples/s]

Map:   0%|          | 0/5560 [00:00<?, ? examples/s]

Map:   0%|          | 0/5560 [00:00<?, ? examples/s]

Token indices sequence length is longer than the specified maximum sequence length for this model (717 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [53, 3188, 26, 340, 1682, 2510, 2621, 10097, 292, 8317, 1083, 25983, 14, 199, 7148, 26, 23593, 12, 1385, 360, 2621, 1908, 13, 8344, 292, 8317, 369, 25983, 26, 199, 17, 14, 8509, 1520, 6611, 26, 2176, 4353, 8627, 1520, 6611, 287, 14200, 288, 16341, 400, 47017, 6923, 5126, 273, 2235, 13101, 14, 199, 18, 14, 6551, 16100, 4635, 26, 2176, 2269, 30646, 8648, 2868, 4197, 5126, 12, 3213, 12, 21638, 3389, 12, 15282, 8156, 12, 287, 3058, 15959, 369, 4214, 579, 16808, 348, 2235, 13101, 14, 199, 19, 14, 6551, 12420, 1104, 7274, 26, 2176, 4353, 2948, 6465, 7274, 2868, 4197, 5126, 611, 4039, 2868, 12, 684, 27755, 348, 3223, 10839, 12, 287, 7356, 2889, 1083, 2235, 13101, 369, 8179, 2690, 287, 368, 41922, 14, 199, 20, 14, 7731, 34571, 26, 2176, 4353, 2716, 34571, 5795, 10845, 1179, 319, 2868, 4197, 531, 369, 2948, 7241, 287, 3111, 2868, 4197, 5126, 14, 199, 21, 14, 24623, 8648, 26, 42479, 8648, 10845, 1179, 287, 16556, 288, 7441, 14082, 7111, 47017, 6923, 5126, 1649, 369, 2948, 11076, 10

In [4]:
for name, param in model.named_parameters():
    print(name)


transformer.wte.weight
transformer.wpe.weight
transformer.h.0.ln_1.weight
transformer.h.0.ln_1.bias
transformer.h.0.attn.c_attn.weight
transformer.h.0.attn.c_attn.bias
transformer.h.0.attn.c_proj.weight
transformer.h.0.attn.c_proj.bias
transformer.h.0.ln_2.weight
transformer.h.0.ln_2.bias
transformer.h.0.mlp.c_fc.weight
transformer.h.0.mlp.c_fc.bias
transformer.h.0.mlp.c_proj.weight
transformer.h.0.mlp.c_proj.bias
transformer.h.1.ln_1.weight
transformer.h.1.ln_1.bias
transformer.h.1.attn.c_attn.weight
transformer.h.1.attn.c_attn.bias
transformer.h.1.attn.c_proj.weight
transformer.h.1.attn.c_proj.bias
transformer.h.1.ln_2.weight
transformer.h.1.ln_2.bias
transformer.h.1.mlp.c_fc.weight
transformer.h.1.mlp.c_fc.bias
transformer.h.1.mlp.c_proj.weight
transformer.h.1.mlp.c_proj.bias
transformer.h.2.ln_1.weight
transformer.h.2.ln_1.bias
transformer.h.2.attn.c_attn.weight
transformer.h.2.attn.c_attn.bias
transformer.h.2.attn.c_proj.weight
transformer.h.2.attn.c_proj.bias
transformer.h.2.ln_2

In [5]:
from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig, TaskType

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=64,
    lora_alpha=128,
    target_modules=["c_attn", "c_proj", "mlp.c_fc", "mlp.c_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


/usr/local/lib/python3.11/dist-packages/peft/tuners/lora/layer.py:1768: UserWarning: fan_in_fan_out is set to False but the target module is `Conv1D`. Setting fan_in_fan_out to True.
  warnings.warn(


trainable params: 9,437,184 || all params: 133,876,992 || trainable%: 7.0491


In [6]:
from transformers import DataCollatorForLanguageModeling
from transformers import TrainingArguments, Trainer

data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

training_args = TrainingArguments(
    output_dir="./gpt2-chatbot-lora",
    num_train_epochs=3,
    per_device_train_batch_size=16,
    gradient_accumulation_steps=2,
    logging_steps=25,
    save_strategy="epoch",
    save_total_limit=1,
    learning_rate=2e-4,
    warmup_ratio=0.03,
    fp16=True,
    lr_scheduler_type="cosine",
    weight_decay=0.01,
    report_to="none"
)

train_dataset = tokenized_dataset["train"]

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator,
)

trainer.train()

No label_names provided for model class `PeftModelForCausalLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


Step,Training Loss
25,3.339700
50,3.293600
75,3.260200
100,3.273800
125,3.254500
150,3.243600
175,3.237200
200,3.136100
225,3.154800
250,3.148900


TrainOutput(global_step=522, training_loss=3.1640855635719736, metrics={'train_runtime': 1214.7308, 'train_samples_per_second': 13.731, 'train_steps_per_second': 0.43, 'total_flos': 4841921461616640.0, 'train_loss': 3.1640855635719736, 'epoch': 3.0})

In [7]:
model.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")
tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-adapter")


('/content/drive/MyDrive/gpt2-lora-adapter/tokenizer_config.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/special_tokens_map.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/vocab.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/merges.txt',
 '/content/drive/MyDrive/gpt2-lora-adapter/added_tokens.json',
 '/content/drive/MyDrive/gpt2-lora-adapter/tokenizer.json')

In [8]:
from peft import PeftModel

if isinstance(model, PeftModel):

    merged_model = model.merge_and_unload()

    merged_model.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
    tokenizer.save_pretrained("/content/drive/MyDrive/gpt2-lora-merged")
else:
    print("tidak ada adapter untuk digabungkan")


In [4]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_path = "/content/drive/MyDrive/gpt2-lora-merged"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForCausalLM.from_pretrained(model_path).to("cpu")

prompt = "User: Siapa presiden pertama Indonesia?\nAI:"
inputs = tokenizer(prompt, return_tensors="pt").to("cpu")

with torch.no_grad():
    outputs = model.generate(
        **inputs,
        max_new_tokens=128,
        do_sample=True,
        temperature=0.6,
        top_p=0.95,
        repetition_penalty=1.2,
        pad_token_id=tokenizer.eos_token_id
    )

print(tokenizer.decode(outputs[0], skip_special_tokens=True))


User: Siapa presiden pertama Indonesia?
AI: Presiden pertama Indonesia adalah Soekarno. Sukarno dikenal sebagai seorang pemimpin politik dan memiliki pengaruh yang signifikan dalam sejarah negara tersebut, serta perannya dalam membentuk pemerintahan di seluruh wilayah nusantara selama periode awal kemerdekaan. Dia terkenal karena upayanya untuk mempertahankan sistem demokrasi parlementer (NSD) dari tangan kolonial Belanda pada tahun 1945-1946, ketika ia mencoba menciptakan parlemen baru dengan kursi parlemen. Dia juga mendukung pendirian partai nasionalis (Partai Sosialis Indonesia). Ia menjadi wakil perdana menteri sementara setelah Jepang menyerah kepada sekutu pada Maret 1946.
Pada akhir masa jabatannya, dia telah menjabat sebagai sekretaris jenderal Dewan Perwakilan Rakyat Republik Indonesia (DPR RI), dan diangkat kembali sebagai Wakil Perdana Menteri pada tanggal 16 Februari 1950
